# PCA 
Data: all years, GEO, GTO, Followup
1) Load data from *.det files, filter data
2) Standardize data and run PCA
3) Plot the cumulative explained variance to show many components capture at least 5% of the variance. 
4) Track the top 3 features in the first two principal components PC1 and PC2, count how often each of the original features appears across all years and orbit types. 
5) Make scatter plots with PC1/PC2. Make pairplots with original features. 

In [ ]:
import numpy as np
import os
import sys
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Load helper
path_to_parent = os.path.abspath("..")
sys.path.append(path_to_parent)
import getdata

# Settings
orbits = ["geo", "gto", "fol"]
years = np.arange(2002, 2024)

plot_dir = "Images/pca"
os.makedirs(plot_dir, exist_ok=True)
getdata.clear_directory(plot_dir)

# PCA analysis and plotting

# Track feature importance frequency
from collections import Counter
importance_counter = Counter()

for orbit in orbits:
    for year in years:
        try:
            # Build input filename
            yy = str(year)[2:]
            if year == 2023:
                file = os.path.join("..", "input", f"stat_Master_23_{orbit}_s1_10cm.det")
            else:
                file = os.path.join("..", "input", f"stat_Master_{yy}_{orbit}_s1.det")

            data = np.array(getdata.array_extender(file)).T

            # Feature indices and names
            orbit_feature_indices = [1, 8, 9, 10, 11, 12, 13, 20]
            feature_names = [
                "diameter", "sem_major", "inc", "ecc", "arg_per", "raan", "true_lat", "mag_obj"
            ]

            # Build DataFrame and adjust RAAN
            df = pd.DataFrame(data[:, orbit_feature_indices], columns=feature_names)
            df['raan'] = ((df['raan'] + 180) % 360) - 180

            # Apply filters
            df_filtered = df[
                (df['mag_obj'] >= 14.5) & (df['mag_obj'] <= 19) &
                (df['sem_major'] < 60000) &
                (df['inc'] < 22) &
                (df['diameter'] > 0.1)
            ].reset_index(drop=True)

            if len(df_filtered) < 2:
                print(f"[{orbit.upper()} {year}] Skipped: only {len(df_filtered)} samples after filtering")
                continue

            # Standardize data
            scaler = StandardScaler()
            scaled = scaler.fit_transform(df_filtered)

            # Perform PCA
            n_comp = min(scaled.shape)
            pca = PCA(n_components=n_comp)
            components = pca.fit_transform(scaled)
            explained_variance = pca.explained_variance_ratio_

            # Plot cumulative variance
            cum_var = np.cumsum(explained_variance)
            plt.figure()
            plt.plot(cum_var, marker='o')
            plt.axhline(0.95, linestyle='--', color='r')
            plt.xlabel('Number of Principal Components')
            plt.ylabel('Cumulative Explained Variance')
            plt.title(f'PCA Explained Variance ({orbit.upper()}, {year})')
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(os.path.join(plot_dir, f"cumvar_{orbit}_{year}.png"))
            plt.close()

            # Determine number of PCs capturing 95% variance
            num_pc = np.argmax(cum_var >= 0.95) + 1
            print(f"[{orbit.upper()} {year}] PCs for 95% var: {num_pc}")

            # Prepare top-3 loadings for PC1 and PC2
            legends = []
            for comp_idx in [0, 1]:
                loadings = np.abs(pca.components_[comp_idx])
                feat_loads = list(zip(feature_names, loadings))
                # record top feature names in global counter
                top3 = sorted(feat_loads, key=lambda x: x[1], reverse=True)[:3]
                for name, _ in top3:
                    importance_counter[name] += 1
                labels = [f"{name} ({coef:.2f})" for name, coef in top3]
                legends.append(f"PC{comp_idx+1}: " + ", ".join(labels))

            # Single scatter plot with combined legend
            plt.figure(figsize=(8, 6))
            plt.scatter(components[:, 0], components[:, 1], s=5, alpha=0.7)
            plt.xlabel('PC1')
            plt.ylabel('PC2')
            plt.title(f'PC1 vs PC2 ({orbit.upper()}, {year})')
            for i, text in enumerate(legends):
                plt.text(0.05, 0.95 - i*0.05, text, transform=plt.gca().transAxes,
                         fontsize='small', va='top')
            plt.grid(which='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()
            plt.savefig(os.path.join(plot_dir, f"scatter_pc1_pc2_{orbit}_{year}.png"))
            plt.close()

            # Pairplot of features
            g = sns.PairGrid(df_filtered, diag_sharey=False)
            g.map_upper(sns.scatterplot, s=10)
            g.map_diag(sns.histplot, kde=False)
            for i, y_var in enumerate(feature_names):
                for j, x_var in enumerate(feature_names):
                    ax = g.axes[i, j]
                    if j < i:
                        ax.set_visible(False)
                    else:
                        ax.set_xlabel(x_var)
                        ax.set_ylabel(y_var)
            g.fig.subplots_adjust(wspace=0.5, hspace=0.5)
            g.fig.suptitle(f'Pairplot ({orbit.upper()}, {year})', y=1.02)
            g.fig.tight_layout()
            g.fig.savefig(os.path.join(plot_dir, f"pairplot_{orbit}_{year}.png"))
            plt.close(g.fig)

        except Exception as e:
            print(f"[{orbit.upper()} {year}] Failed: {e}")

# After all runs, print most frequently important features
print("Feature importance frequency across PC1/PC2 top-3:")
for feature, count in importance_counter.most_common():
    print(f"  {feature}: {count}")

[GEO 2002] PCs for 95% var: 6
[GEO 2003] PCs for 95% var: 6
[GEO 2004] PCs for 95% var: 6
[GEO 2005] PCs for 95% var: 6
[GEO 2006] PCs for 95% var: 6
[GEO 2007] PCs for 95% var: 6
[GEO 2008] PCs for 95% var: 5
[GEO 2009] PCs for 95% var: 6
[GEO 2010] PCs for 95% var: 6
[GEO 2011] PCs for 95% var: 6
[GEO 2012] PCs for 95% var: 5
[GEO 2013] Skipped: only 1 samples after filtering
[GEO 2014] PCs for 95% var: 4
[GEO 2015] PCs for 95% var: 6
[GEO 2016] PCs for 95% var: 6
[GEO 2017] PCs for 95% var: 6
[GEO 2018] PCs for 95% var: 6
[GEO 2019] PCs for 95% var: 6
[GEO 2020] PCs for 95% var: 6
[GEO 2021] PCs for 95% var: 6
[GEO 2022] PCs for 95% var: 6
[GEO 2023] PCs for 95% var: 6
[GTO 2002] PCs for 95% var: 6
[GTO 2003] PCs for 95% var: 6
[GTO 2004] PCs for 95% var: 6
[GTO 2005] PCs for 95% var: 6
[GTO 2006] PCs for 95% var: 6
[GTO 2007] PCs for 95% var: 6
[GTO 2008] PCs for 95% var: 6
[GTO 2009] PCs for 95% var: 6
[GTO 2010] PCs for 95% var: 6
[GTO 2011] PCs for 95% var: 6
[GTO 2012] PCs for 

# PCA dominant features: loadings and over time  
1) Load and filter detection data. 
2) Standardize data and run PCA. 
3) Record the fraction of the variance explaind by PC1 and the feature loadings for PC1. 
4) Store results in dataframe and crease plots:   
    a) How much of the variance PC1 explains over time (one plot point per year), make three curves for GEO, GTO and Followup.   
    b) Heatmaps of the PC1 feature loadings over time for each orbit type

In [36]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

YEARS       = list(range(2002, 2024))
ORBIT_TYPES = ["geo", "gto", "fol"]
BASE_DIR    = os.path.join("..", "input")

STORE_FOLDER = "Images\pca_and_pairplots"
os.makedirs(STORE_FOLDER, exist_ok=True)
getdata.clear_directory(STORE_FOLDER)

# indices & feature names
ORBIT_IDX = [1, 8, 9, 10, 11, 12, 13, 20]
FEATURES = ["diameter", "sem_major", "inc", "ecc", "arg_per", "raan", "true_lat", "mag_obj"]

# Clear and create storage folder
getdata.clear_directory(STORE_FOLDER)
os.makedirs(STORE_FOLDER, exist_ok=True)

# Fundamental filters
def apply_filters(df):
    df = df.copy()
    df['raan'] = ((df['raan'] + 180) % 360) - 180
    return df[
        (df.mag_obj.between(14.5, 19)) &
        (df.sem_major < 60000) &
        (df.inc < 22) &
        (df.diameter > 0.1)
    ]

# PCA runner
def run_pca_for(year: int, orbit: str):
    year2 = str(year)[2:]
    if year == 2023:
        path = os.path.join(BASE_DIR, f"stat_Master_23_{orbit}_s1_10cm.det").replace("\\", "/")
    else:
        path = os.path.join(BASE_DIR, f"stat_Master_{year2}_{orbit}_s1.det").replace("\\", "/")

    raw = np.array(getdata.array_extender(path)).T
    df = pd.DataFrame(raw[:, ORBIT_IDX], columns=FEATURES)
    df_f = apply_filters(df)
    X = StandardScaler().fit_transform(df_f)

    n_comp = min(X.shape[0], X.shape[1])
    pca = PCA(n_components=n_comp).fit(X)

    cumvar = np.cumsum(pca.explained_variance_ratio_)
    n95 = np.searchsorted(cumvar, 0.95) + 1
    load1 = pca.components_[0]

    return {
        "year": year,
        "orbit": orbit,
        "n_samples": X.shape[0],
        "n95": n95,
        "pc1_var": cumvar[0],
        **{f"load1_{f}": load for f, load in zip(FEATURES, load1)}
    }

# Collect results
results = []
for orb in ORBIT_TYPES:
    for yr in YEARS:
        try:
            meta = run_pca_for(yr, orb)
            results.append(meta)
        except Exception as e:
            print(f"Failed {orb} {yr}: {e}")

df_res = pd.DataFrame(results)

# Plot: PC1 variance ratio over time
plt.figure(figsize=(8, 5))
sns.lineplot(data=df_res, x="year", y="pc1_var", hue="orbit", marker="o")
plt.title("Fraction of Variance explained by PC1 over time")
plt.ylabel("PC1 Variance Ratio")
plt.tight_layout()
plt.grid(True)
plt.savefig(os.path.join(STORE_FOLDER, "pc1_variance_over_time.png"))
plt.close()

# Plot: PC1 loadings heatmap for each orbit
for orb in ORBIT_TYPES:
    pivot = (df_res[df_res.orbit == orb]
             .set_index("year")[[f"load1_{f}" for f in FEATURES]])
    plt.figure(figsize=(6, 4))
    sns.heatmap(pivot, center=0, cmap="coolwarm", annot=True, fmt=".2f")
    plt.title(f"PC1 Loadings ({orb.upper()})")
    plt.tight_layout()
    filename = f"pc1_loadings_{orb}.png"
    plt.savefig(os.path.join(STORE_FOLDER, filename))
    plt.close()


<>:13: SyntaxWarning: invalid escape sequence '\p'
<>:13: SyntaxWarning: invalid escape sequence '\p'
C:\Users\fionu\AppData\Local\Temp\ipykernel_19620\3618191254.py:13: SyntaxWarning: invalid escape sequence '\p'
  STORE_FOLDER = "Images\pca_and_pairplots"
c:\Users\fionu\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:591: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\fionu\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:591: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
